# High-level CNTK Example

In [1]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Convolution2D, MaxPooling, Dense, Dropout
from common.params import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
CNTK:  2.0rc1


In [3]:
def create_lenet():
    with cntk.layers.default_options(init = cntk.glorot_normal(), activation = cntk.relu):
        x = Convolution2D(filter_shape=(3, 3), num_filters=48, pad=True)(features)
        x = Convolution2D(filter_shape=(3, 3), num_filters=48)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)

        x = Convolution2D(filter_shape=(3, 3), num_filters=96, pad=True)(x)
        x = Convolution2D(filter_shape=(3, 3), num_filters=96)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)
        
        x = Convolution2D(filter_shape=(3, 3), num_filters=192, pad=True)(x)
        x = Convolution2D(filter_shape=(3, 3), num_filters=192)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)        
        
        x = Dense(512)(x)
        x = Dropout(0.5)(x)
        x = Dense(N_CLASSES, activation=None)(x)
        return x

In [4]:
def init_model(m):
    # Loss
    loss = cntk.cross_entropy_with_softmax(m, labels)
    # Classification error
    label_error = cntk.classification_error(m, labels)
    # Init trainer object
    lr_schedule = cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) 
    m_schedule = cntk.momentum_schedule(MOMENTUM)
    # Where can I say no wd and no nesterov; where is momentum
    learner = cntk.momentum_sgd(m.parameters, lr=lr_schedule, momentum=m_schedule)
    trainer = cntk.Trainer(m, (loss, label_error), [learner])
    return trainer

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=True, one_hot=True)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
# CNTK format
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 float32 float32
CPU times: user 3.05 s, sys: 1.38 s, total: 4.43 s
Wall time: 21.1 s


In [6]:
%%time
# Placeholders
features = cntk.input_variable((3, 32, 32), np.float32)
labels = cntk.input_variable(N_CLASSES, np.float32)
# Load symbol
sym = create_lenet()

CPU times: user 28 ms, sys: 208 ms, total: 236 ms
Wall time: 610 ms


In [7]:
%%time
trainer = init_model(sym)

CPU times: user 36 ms, sys: 576 ms, total: 612 ms
Wall time: 2.2 s


In [8]:
%%time
# Train model
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        trainer.train_minibatch({features: data, labels: label})
    # Log (this is just last batch in epoch, not average of batches)
    eval_error = trainer.previous_minibatch_evaluation_average
    print("Epoch %d  |  Accuracy: %.6f" % (j+1, (1-eval_error)))

Epoch 1  |  Accuracy: 0.140625
Epoch 2  |  Accuracy: 0.171875
Epoch 3  |  Accuracy: 0.265625
Epoch 4  |  Accuracy: 0.421875
Epoch 5  |  Accuracy: 0.437500
Epoch 6  |  Accuracy: 0.437500
Epoch 7  |  Accuracy: 0.484375
Epoch 8  |  Accuracy: 0.531250
Epoch 9  |  Accuracy: 0.546875
Epoch 10  |  Accuracy: 0.531250
Epoch 11  |  Accuracy: 0.531250
Epoch 12  |  Accuracy: 0.609375
Epoch 13  |  Accuracy: 0.468750
Epoch 14  |  Accuracy: 0.640625
Epoch 15  |  Accuracy: 0.656250
Epoch 16  |  Accuracy: 0.640625
Epoch 17  |  Accuracy: 0.656250
Epoch 18  |  Accuracy: 0.703125
Epoch 19  |  Accuracy: 0.625000
Epoch 20  |  Accuracy: 0.640625
CPU times: user 4min 56s, sys: 45.9 s, total: 5min 42s
Wall time: 10min 25s


In [9]:
%%time
# Predict and then score accuracy
# Apply softmax since that is only applied at training
# with cross-entropy loss
z = cntk.softmax(sym)
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = np.argmax(y_test[:n_samples], axis=-1)
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    predicted_label_probs = z.eval({features : data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = np.argmax(predicted_label_probs, axis=-1)
    c += 1

CPU times: user 960 ms, sys: 336 ms, total: 1.3 s
Wall time: 2.69 s


In [10]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.681991185897
